# Coursework 4

### PLAGIARISM/COLLUSION DECLARATION

 

Coursework submitted for assessment must be the original work of you and your group. Assignments are subjected to regular checks for plagiarism and/or collusion. Plagiarism is the presentation of another person’s thoughts or words (those outside your group) as if they were your own. Collusion involves obtaining help from someone outside your group to complete your work. In preparing your coursework, you should not seek help, or copy from any other person or source, including the

Internet, without proper and explicit acknowledgement.

 

There is a procedure in place for you to declare individual contributions within your group for courswork. You must declare the contributions fairly and accurately.

 

You must not disclose your solutions or insights related to coursework with anyone else, including future students or the Internet.

 

By acknowledging the the statements above, you are declaring that both this and all subsequent pieces of coursework are, and will remain, the original work of you and your group.

 

* Submissions will not be accepted without the aforementioned declaration.

 

* Members of a group are deemed to have collective responsibility for the integrity for work submitted and are liable for any penalty imposed, proportionate to their contributions.

In [1]:
using JLD2,FileIO

In [2]:
Base.@kwdef mutable struct CourseWork_4
    
    CID::Int
    Plagiarism_declare::Union{Missing,Bool}
    
    # 11. MRI CS Recovery: ADMM
    
    Q11_data_Z1::Matrix
    Q11_data_Z2::Matrix
    Q11_data_U1::Matrix
    Q11_data_U2::Matrix
    Q11_data_Omega::Matrix
    Q11_data_y::Vector
    Q11_data_lambda_1::Float64
    Q11_data_lambda_2::Float64
    Q11_data_rho::Float64
    
    Q11_1_ans_X::Union{Missing,Matrix}
    Q11_2_ans_Z1::Union{Missing,Matrix}
    Q11_2_ans_Z2::Union{Missing,Matrix}
    Q11_3_ans_U1::Union{Missing,Matrix}
    Q11_3_ans_U2::Union{Missing,Matrix}
    Q11_4_ans_R::Union{Missing,Matrix}
    Q11_5_ans_S::Union{Missing,Matrix}
    Q11_6_ans_X::Union{Missing,Matrix}
    
    # 12. Blind Deconvolution: Convex Relaxation and ADMM
    
    Q12_data_Z1::Matrix
    Q12_data_Z2::Matrix
    Q12_data_U1::Matrix
    Q12_data_U2::Matrix
    Q12_data_y::Vector
    Q12_data_lambda_1::Float64
    Q12_data_lambda_2::Float64
    Q12_data_rho::Float64
    
    Q12_1_ans_X::Union{Missing,Matrix}
    Q12_2_ans_Z1::Union{Missing,Matrix}
    Q12_2_ans_Z2::Union{Missing,Matrix}
    Q12_3_ans_U1::Union{Missing,Matrix}
    Q12_3_ans_U2::Union{Missing,Matrix}
    Q12_4_ans_R::Union{Missing,Matrix}
    Q12_5_ans_S::Union{Missing,Matrix}
    Q12_6_ans_X::Union{Missing,Matrix}
    Q12_7_ans_s::Union{Missing,Vector}
    Q12_7_ans_h::Union{Missing,Vector}
    
    # 13. Neural Network Training: Block Coordinate Descent
    
    Q13_data_Y::Matrix
    Q13_data_X::Matrix
    Q13_data_Ytest::Matrix
    Q13_data_Xtest::Matrix
    Q13_data_X1::Matrix
    Q13_data_Z1::Matrix
    Q13_data_Z2::Matrix
    Q13_data_alpha::Float64
    
    Q13_1_ans_W1::Union{Missing,Matrix}
    Q13_1_ans_W2::Union{Missing,Matrix}
    Q13_2_ans_X1::Union{Missing,Matrix}
    Q13_3_ans_Z1::Union{Missing,Matrix}
    Q13_3_ans_Z2::Union{Missing,Matrix}
    Q13_4_ans_g::Union{Missing,Matrix}
    Q13_5_ans_f::Union{Missing,Vector}
    Q13_5_ans_gx::Union{Missing,Vector}
    Q13_5_ans_l::Union{Missing,Vector}
    Q13_5_ans_ltest::Union{Missing,Vector}
    
    
end

CourseWork_4

In [3]:

A = JLD2.load("Banthia Dev2016674StudentFile (5).jld2");

A = A[ "CourseWork_4" ];

In [5]:
using Pkg
Pkg.add("FFTW")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [33]:
using LinearAlgebra
using FFTW

omega = A.Q11_data_Omega

Z1 = A.Q12_data_Z1
Z2 = A.Q12_data_Z2
U1 = A.Q12_data_U1
U2 = A.Q12_data_U2
y = A.Q12_data_y
lambda1 = A.Q12_data_lambda_1
lambda2 = A.Q12_data_lambda_2
rho = A.Q12_data_rho

function create_sparse_row_vector(m, n, k)

    indices = []

    for i in 1:m  
        j = k + 1 - i 
        if j >= 1 && j <= n 
            push!(indices, i)  
        end
    end

    total_size = m * n  
    sparse_vector = zeros(Int, 1, total_size)

    for i in indices
        col_index = (k + 1 - i - 1) * m + i 
        if col_index >= 1 && col_index <= total_size 
            sparse_vector[1, col_index] = 1
        end
    end

    return sparse_vector
end


function convolution_matrix(m,n)

    conv = zeros(m, m*n) 
    for k in 1:(m)
        sparse_row = create_sparse_row_vector(m, n, k)
        conv[k,:] = sparse_row

    end

    return conv
end

function X_next(conv, rho, y, U1, Z1, U2, Z2)

    temp = conv' * conv
    row_num = size(temp)[1]
    col_num = size(temp)[2]
    scaling_matrix = pinv(temp + rho*Matrix(I, row_num, col_num) )

    return scaling_matrix * ((conv' * y) + vec(U1) + vec(rho*Z1) + vec(idct(U2)) + vec(idct(Z2)))


end

function singular_value_thresholding(Z1, tau)

    tau = tau*20

    U, S, V = svd(Z1)
    println(S)
    S_thresholded = max.(S .- tau, 0)
    return U * Diagonal(S_thresholded) * V'
end

function row_thresholding(X, λ)

    Z = similar(X)
    for i in 1:size(X, 1)
        row = X[i, :]  
        norm_row = norm(row, 2)  
        
        if norm_row <= λ
            Z[i, :] .= 0.0
        else
            Z[i, :] .= (1 - λ / norm_row) * row
        end
    end
    
    return Z
end


function ADMM(conv, rho, y, U1, Z1, U2, Z2)

    for i in 1:100


        Z1_prev = Z1
        Z2_prev = Z2

        X = X_next(conv, rho, y, U1, Z1, U2, Z2)
        X = reshape(X, (m,n))

        z1_threshold =  lambda1/rho
        z1_target_matrix = X - U1/rho
        Z1 = singular_value_thresholding(z1_target_matrix, z1_threshold)

        z2_threshold = lambda2/rho
        z2_target_matrix = dct(X) - U2/rho
        Z2 = row_thresholding(z2_target_matrix, z2_threshold)

        r1 = Z1 - X
        r2 = Z2 - dct(X)
        primal_residual = sqrt(norm(r1, 2)^2 + norm(r2, 2)^2)
    
        s1 = rho * (Z1 - Z1_prev)
        s2 = rho * idct(Z2 - Z2_prev)
        dual_residual = sqrt(norm(s1, 2)^2 + norm(s2, 2)^2)

        # println(norm(y-conv*vec(X)))
        # println(dual_residual)



        U1 = U1 + rho*(Z1 - X)
        U2 = U2 + rho*(Z2 - dct(X))


    end

    return X

end


m = size(U1)[1]
n = size(U1)[2]
conv = convolution_matrix(m,n)
X = X_next(conv, rho, y, U1, Z1, U2, Z2)
X = reshape(X, (m,n))

z1_threshold =  lambda1/rho
z1_target_matrix = X - U1/rho
Z1_updated = singular_value_thresholding(z1_target_matrix, z1_threshold)

z2_threshold = lambda2/rho
z2_target_matrix = dct(X) - U2/rho
Z2_updated = singular_value_thresholding(z2_target_matrix, z2_threshold)

U1_updated = U1 + rho*(Z1_updated - X)
U2_updated = U2 + rho*(Z2_updated - dct(X))


X = ADMM(conv, rho, y, U1, Z1, U2, Z2)



[27.729022039470532, 27.521179023360098, 26.32859323811591, 25.04207811716896, 24.12316144549163, 23.12446163485339, 22.883554209558465, 21.595063837638676, 20.055917860135885, 19.466299954763866]
[26.920867400920184, 25.996743428795188, 25.84336878709763, 24.328504716657946, 23.579361450373412, 22.44079732509082, 21.534235668208716, 20.48783254572052, 19.688724533108665, 18.322004936992943]
0.1
[27.729022039470532, 27.521179023360098, 26.32859323811591, 25.04207811716896, 24.12316144549163, 23.12446163485339, 22.883554209558465, 21.595063837638676, 20.055917860135885, 19.466299954763866]
[46.23469413350124, 44.98647862406294, 44.06847478543418, 39.97286413312843, 39.439746037996784, 38.619276430328476, 37.12175253316925, 35.73942967935087, 33.86185201421841, 32.99355466782631]
[88.09761329732281, 85.46993354762193, 83.74218847484222, 75.57488109742825, 74.5939187390642, 73.01275624354463, 70.02680082939857, 67.31138086955691, 63.57867316264475, 61.98960278362409]


InterruptException: InterruptException:

In [25]:
using Images
using FileIO
using Plots


p1 = plot(heatmap(y, color=:grays), title="Original Image", legend=false)


plot(p1)

In [17]:
using LinearAlgebra

y = A.Q13_data_Y
X = A.Q13_data_X
y_test = A.Q13_data_Ytest
X_test = A.Q13_data_Xtest
X1 = A.Q13_data_X1
Z1 = A.Q13_data_Z1
Z2 = A.Q13_data_Z2
alpha = A.Q13_data_alpha

alpha = 0.5


function update_W1(Z1, X)

    W1 = alpha*(Z1*X')*pinv(X*X')
    grad_W1 = norm(alpha*(Z1 - W1*X)*X')

    return W1, grad_W1
end

function update_W2(Z2, X1)
    W2 = alpha*(Z2*X1')*pinv(X1*X1')
    grad_W2 = norm(alpha*(Z2 - W2*X1)*X1')

    return W2, grad_W2
end

function update_X1(Z1)
    return  max.(Z1, 0)
end



function proximal_relu_matrix(a::AbstractMatrix, b::AbstractMatrix, λ::Real)

    proximal_element(a_ij, b_ij) = begin
        term = a_ij + λ * b_ij
        threshold = -1*(sqrt(λ * (λ + 1)) - λ) * a_ij

        if (term >= 0 && b_ij >= 0) || ((threshold <= λ * b_ij)  && (λ * b_ij < 0))
        
            return (a_ij + λ * b_ij) / (1 + λ)

        elseif (-a_ij <= λ * b_ij) && (λ * b_ij < threshold) && (threshold < 0)
            return b_ij

        elseif term < 0
            return min(b_ij, 0)
        else
            error("Unexpected case encountered at element ($a_ij, $b_ij)")
        end
    end
    return map(proximal_element, a, b)
end


W1, grad_W1 = update_W1(Z1, X)
W2, grad_W2 = update_W2(Z2, X1)
X1 = update_X1(Z1)

a = y
b = W2*X1

Z2 = proximal_relu_matrix(a::AbstractMatrix, b::AbstractMatrix, alpha)

a = X1
b = W1*X

Z1 = proximal_relu_matrix(a::AbstractMatrix, b::AbstractMatrix, alpha)


function BCD(Z1, X, Z2, X1, W1, W2, alpha, y)

    for i in 1:100
        println(norm(y-max.(Z2, 0)))
        W1, grad_W1 = update_W1(Z1, X)
        W2, grad_W2 = update_W1(Z2, X1)
        X1 = update_X1(Z1)
        a = y
        b = W2*X1
        Z2 = proximal_relu_matrix(a, b, alpha)
        a = X1
        b = W1*X
        Z1 = proximal_relu_matrix(a::AbstractMatrix, b::AbstractMatrix, alpha)

    end

end

BCD(Z1, X, Z2, X1, W1, W2, alpha, y)





1787.0574387205406
1758.4241081214223
1754.3306006914963
1753.2116071506546
1752.5681109484085
1752.0896240020224
1751.6891537523634
1751.3599958857299
1751.0937788216752
1750.8757985762115
1750.7002984574049
1750.5707836529734
1750.466518292518
1750.394089098106
1750.3378924967217
1750.2972985668723
1750.2741052782023
1750.2546992003702
1750.2465973106782
1750.246743000866
1750.2519565046011
1750.2616095122842
1750.2683539656912
1750.2798934926138
1750.2920311697649
1750.3099771860823
1750.3254140005893
1750.3383672450225
1750.3512356518172
1750.370814022121
1750.3848322452407
1750.3995309470868
1750.4125137555445
1750.4225878386362
1750.4367019591398
1750.4524211121775
1750.4680161876454
1750.4791751856858
1750.4908843431933
1750.505488193123
1750.5187665246808
1750.533036782872
1750.5437480234548
1750.5538545837524
1750.5608627672493
1750.5718263000713
1750.5827362139453
1750.5953250632049
1750.6062947707583
1750.6181180724625
1750.6289304462284
1750.6383738906268
1750.648522106335
